In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix
from nltk.stem.snowball import SnowballStemmer
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import seaborn as sn
from sklearn import datasets
from nltk.tokenize import RegexpTokenizer
from gensim.sklearn_api.w2vmodel import W2VTransformer
from gensim.sklearn_api.d2vmodel import D2VTransformer
from nltk.corpus import stopwords
from ast import literal_eval
import xlrd
import openpyxl

c:\users\m6800\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\m6800\appdata\local\programs\python\python36\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def row(key, scores, params):
            d = {
                 'estimator': key,
                 'mean_score': np.mean(scores),
                 'f1-scores': scores,
            }
            return pd.Series({**params,**d})
tokenizer = RegexpTokenizer(r'\w+')

In [3]:
# Importing data
df4 = pd.read_csv("Handelingen.csv", index_col=0)
df4 = df4.loc[df4['speech category'] == 'Main Speech']
stemmer = SnowballStemmer("dutch")

## FIX THIS INTO ONE LINE
list_text = df4.text.apply(lambda x: [stemmer.stem(t) for t in tokenizer.tokenize(x)])
normal_text = df4.text.apply(lambda x: ' '.join([stemmer.stem(t) for t in tokenizer.tokenize(x)]))
df4.head(10)

,surname,party,text,file,speech category,date,tags
107136,Wilders,PVV,\r\n \r\n Mevrouw de voo...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"['Bestuur | Parlement', 'Financiën | Begroting']"
107137,Roemer,SP,\r\n \r\n Voorzitter. Vo...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"['Bestuur | Parlement', 'Financiën | Begroting']"
107138,Pechtold,D66,\r\n \r\n Voorzitter. Tw...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"['Bestuur | Parlement', 'Financiën | Begroting']"
107139,Van Haersma Buma,CDA,\r\n \r\n Voorzitter. Al...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"['Bestuur | Parlement', 'Financiën | Begroting']"
107148,Thieme,PvdD,\r\n \r\n Voorzitter. We...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"['Bestuur | Parlement', 'Financiën | Begroting']"
107149,Slob,ChristenUnie,"\r\n \r\n Voorzitter, ik...",h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"['Bestuur | Parlement', 'Financiën | Begroting']"
107154,Samsom,PvdA,\r\n \r\n Voorzitter. De...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"['Bestuur | Parlement', 'Financiën | Begroting']"
107272,Van Ojik,GroenLinks,\r\n \r\n Voorzitter. Ik...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"['Bestuur | Parlement', 'Financiën | Begroting']"
107278,Slob,ChristenUnie,\r\n \r\n Dat is een dui...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"['Bestuur | Parlement', 'Financiën | Begroting']"
107282,Van der Staaij,SGP,\r\n \r\n Voorzitter. Ge...,h-tk-20122013-100-3.xml,Main Speech,2013-06-26,"['Bestuur | Parlement', 'Financiën | Begroting']"


In [1]:
models_df = pd.read_excel('Models.xlsx', index_col=0)
models_df = models_df.loc[models_df.DOEN == True]
models_dict = {}

for x,y in zip(models_df.Classifier,models_df.PIPELINE):
    exec(compile("a="+y,'','exec'))
    models_dict[x] = Pipeline(a)    
params_dict = {}

for clf in set(models_df.Classifier):
    params_df = models_df.loc[models_df.Classifier == clf]
    params_df = params_df.loc[params_df.DOEN == True]
    params_df = params_df.dropna(axis=1, how='all')
    params_df = params_df.drop([col for col in params_df.columns if "__" not in col], axis =1)
    temp_dict = {}
    for x,y in params_df.to_dict(orient='list'):
        a = literal_eval(y[0])
        temp_dict[x] = a   
    params_dict[clf] = temp_dict
    
text_dict = {x:y for x,y in zip(models_df.Classifier, models_df.RAW)}

NameError: name 'pd' is not defined

In [ ]:
for name,pipe in models.items():
    if rawdict[name]:
         text = list(list_text)
    else:
        text = list(normal_text)
    gs = GridSearchCV(pipe, params[name], cv=5, scoring='f1_weighted')
    gs.fit(text, df4.party)
    rows = [row(name, gsc.cv_validation_scores, gsc.parameters) for gsc in gs.grid_scores_]
    df2 = pd.concat(rows, axis=1).T
    pd.concat([df2,pd.read_csv('Scores.csv', index_col=0)], ignore_index=True).sort_values(['mean_score'], ascending=False).to_csv('Scores.csv')
    df.loc[df.Classifier==name,"DOEN"]=False
    df.to_excel('Models.xlsx')
    
#https://stackoverflow.com/questions/46735847/save-best-params-in-gridsearch-in-a-pandas-dataframe
# https://stackoverflow.com/questions/36271413/pandas-merge-nearly-duplicate-rows-based-on-column-value?rq=1